In [32]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
import torch.nn.functional as F

from efficientnet_pytorch import EfficientNet

In [33]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [34]:
model = EfficientNet.from_pretrained('efficientnet-b0')

model.eval()

# Chuyển đổi hình ảnh đầu vào thành định dạng phù hợp
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)  # Thêm batch dimension

# Dự đoán
def predict(image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
    return output

# Ví dụ sử dụng
image_path = r"D:\SkinCancerPrediction\Preprocessed\HairRemoved\ISIC_0077975.jpg"
output = predict(image_path)
print(output)

Loaded pretrained weights for efficientnet-b0
tensor([[-1.0045e+00,  1.5546e+00,  9.2334e-01, -2.4478e-01,  3.8336e-02,
          2.4337e+00,  1.1894e+00, -3.5873e-01,  2.3623e-01,  1.0093e+00,
         -4.3248e-01, -8.4150e-01,  1.8520e-01, -3.0666e-01, -7.9108e-01,
         -1.1982e+00, -1.0178e+00, -1.1005e+00,  7.6849e-01, -1.8070e-02,
         -5.7447e-01,  3.7023e-01,  5.0308e-02,  5.4371e-01, -9.8261e-01,
         -6.7180e-01,  1.3512e+00,  5.4849e-01, -1.2794e+00,  5.0657e-01,
         -1.3911e+00, -1.4001e+00,  1.4091e+00, -3.4270e-01, -2.2011e-01,
         -3.8965e-01,  8.3143e-01, -3.8336e-01,  8.0976e-01, -1.2195e+00,
         -2.3449e+00,  1.2802e+00, -3.7099e-01,  6.9680e-01, -6.7316e-01,
         -1.4689e+00, -1.7447e+00, -3.8037e-01, -7.6859e-01, -2.2890e+00,
         -2.2510e+00, -2.3794e-01, -7.4080e-01, -1.1728e-01, -2.5566e-01,
         -1.7430e+00, -1.5593e+00, -1.4803e-01, -3.1166e-01, -5.6510e-01,
         -1.3718e-01, -1.3554e+00, -2.6969e-01,  1.9920e+00, -5.72

In [35]:
class SkinCancerDataset(Dataset):
    def __init__(self, metadata, images_dir, transform=None):
        self.metadata = metadata
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.metadata.iloc[idx]['isic_id'] + '.jpg')
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)
        
        label = torch.tensor(self.metadata.iloc[idx]['target'], dtype=torch.float32).unsqueeze(0)    
        return image, label

In [36]:
metadata_path = r"D:\SkinCancerPrediction\Original\isic-2024-challenge\train-metadata.csv"
images_dir = r"D:\SkinCancerPrediction\Model\Test\Image\HairRemoved"

In [37]:
image_names = []

for filename in os.listdir(images_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_names.append(filename)

print(image_names.__len__())


786


In [38]:
metadata = pd.read_csv(metadata_path)

image_names_no_ext = [os.path.splitext(name)[0] for name in image_names]

metadata = metadata[metadata['isic_id'].isin(image_names_no_ext)]
metadata = metadata[['isic_id', 'target']]
metadata

C:\Users\Kah\AppData\Local\Temp\ipykernel_10700\2420353925.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_path)


,isic_id,target
0,ISIC_0015670,0
1,ISIC_0015845,0
2,ISIC_0015864,0
3,ISIC_0015902,0
4,ISIC_0024200,0
...,...,...
396165,ISIC_9877311,1
398671,ISIC_9941636,1
399936,ISIC_9972649,1
399943,ISIC_9972877,1


In [39]:
dataset = SkinCancerDataset(metadata=metadata, images_dir=images_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [40]:
import torch

def test_shapes(data_loader, model, device):
    # Set model to evaluation mode
    model.eval()
    
    # Fetch a single batch
    images, masks = next(iter(data_loader))
    
    # Move data to the specified device
    images = images.to(device)
    masks = masks.to(device)
    
    # Forward pass through the model
    with torch.no_grad():
        outputs = model(images)
    
    # Print shapes
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {masks.shape}")
    print(f"Model output shape: {outputs.shape}")


# Example usage:
if __name__ == "__main__":
    # Assuming you have the following variables defined:
    # - train_loader: your DataLoader
    # - model: your model instance
    # - device: your computing device (cpu or cuda)
    
    test_shapes(dataloader, model, 'cpu')


Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32, 1])
Model output shape: torch.Size([32, 1000])


In [41]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
model.train()

# Định nghĩa tiêu chuẩn và bộ tối ưu hóa
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# Lưu mô hình
torch.save(model.state_dict(), 'efficientnet_model.pth')

Loaded pretrained weights for efficientnet-b0


RuntimeError: 0D or 1D target tensor expected, multi-target not supported